##### <a href="https://colab.research.google.com/github/qazalmehrban/Medical-Segmentation/blob/ghazal/kera_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from skimage.transform import resize
import numpy as np
import nibabel as nib
import os 

import matplotlib.pyplot as plt


In [4]:
heaight, width = 200,200

In [5]:
segment_path = os.path.join(os.getcwd(), 'segmentation') 
img_path = os.path.join(os.getcwd(), 'samples') 

In [6]:
segment_file = sorted(os.listdir(segment_path))
segment_img = sorted(os.listdir(img_path))


In [7]:
def make_box(mask):
    box = []
    obj_ids = np.unique(mask)
    if mask.dtype == bool:
        if len(np.unique(mask))==2:
            
            y_min = np.nonzero(mask)[0].min()
            y_max = np.nonzero(mask)[0].max()
            x_min = np.nonzero(mask)[1].min()
            x_max = np.nonzero(mask)[1].max()
            box.append([x_min, y_min, x_max, y_max])
    else :
        
        for i in  obj_ids[1:]:
            y_min = np.nonzero(mask==i)[0].min()
            y_max = np.nonzero(mask==i)[0].max()
            x_min = np.nonzero(mask==i)[1].min()
            x_max = np.nonzero(mask==i)[1].max()
            box.append([x_min, y_min, x_max, y_max])
    return dict(zip([f"label{int(i)}" for i in obj_ids[1:]],box))

In [9]:
crop_size = 160
p_seg = []
p_im = []
for i, i1 in zip(segment_file, segment_img):
    im = nib.load(os.path.join(img_path, i1))    
    im = np.array(im.dataobj)

    Masks = nib.load(os.path.join(segment_path, i))    
    Masks = np.array(Masks.dataobj)
    Masks.shape
    for  j in range(Masks.shape[2]):
        
        k=make_box(Masks[:,:,j].astype(np.bool))
        if len(k)!=0:
            b_list = k['label1']
            x_min, y_min, x_max, y_max = b_list
            
            length_x = x_max -x_min + 1
            length_y = y_max - y_min + 1
            del_x = crop_size - length_x 
            del_y = crop_size - length_y 

            if del_x%2==0:
                kx = del_x//2
                sx=0
            else:
                kx = del_x//2

                sx = 1
            if del_y%2==0:
                ky = del_y//2
                sy=0
            else:
                ky = del_y//2
                sy = 1
            crop = Masks[ y_min - ky - sy : y_max + ky + 1, x_min - kx - sx : x_max + kx  + 1,j ]
            crop_im = im[ y_min - ky - sy : y_max + ky + 1, x_min - kx - sx : x_max + kx  + 1,j ]
            p_seg.append(crop)
            p_im.append(crop_im)

           
                
            
        
    

In [10]:
segment_numpy = np.stack(p_seg,2)
image_numpy = np.stack(p_im, 2)

In [11]:
train = image_numpy/image_numpy.max()

In [12]:
tra  = np.transpose(train,(2,0,1))

In [13]:
seg = np.transpose(segment_numpy,(2,0,1))

In [ ]:
'''
def gen():
  samples = '/content/drive/MyDrive/samples/'
  patients = next(os.walk(samples))[2]
  patients.sort()
  labels = '/content/drive/MyDrive/segmentation/'
  segmentations =next(os.walk(labels))[2]
  segmentations.sort()
  for i,j in zip(patients,segmentations):
    Images = nib.load((samples + i))
    im = np.array(Images.dataobj)
    Masks = nib.load((labels + j))    
    Masks = np.array(Masks.dataobj)
    img_resize = resize(im,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    mask_resize = resize(Masks,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    for k in range(im.shape[2]):
      im1 = np.expand_dims(img_resize[:,:,k],2)
      ma = np.expand_dims(mask_resize[:,:,k],2)

      yield {'Image':im1, 'Mask':ma}
      
'''

In [ ]:
dat = tf.data.Dataset.from_generator(gen, output_types ={"Image": tf.float32, "Mask": tf.float32}).batch(45)

In [ ]:
it=iter(dat)

In [ ]:
dataset = next(it)

# Model

In [16]:
img_size = (160, 160)

In [17]:


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (1,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size, 3)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 80, 80, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 80, 80, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 80, 80, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

# train

In [ ]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint("oxford_segmentation.h5", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
epochs = 15
model.fit(tra, seg, epochs=epochs)

In [ ]:
import os

In [ ]:
root = "home"

In [ ]:
al_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)
val_preds = model.predict(val_gen)



# Display results for validation image #10
i = 10

# Display input image
display(Image(filename=val_input_img_paths[i]))

# Display ground-truth target mask
img = PIL.ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)

# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.

In [ ]:

def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)


In [ ]:
pr = model.predict(tra[:670])

In [ ]:
mask1 = np.argmax(pr[20], axis=-1)


In [ ]:
plt.imshow(mask1)

In [ ]:
pr[2]